In [1]:
import pandas as pd

# Read in the datasets
noc_regions_data = pd.read_csv("datasets/noc_regions-parsed.csv")
nocs_list = pd.read_csv("datasets/nocs_list.csv")

iso_data = pd.read_csv("datasets/iso_codes.csv")

In [2]:
# get the number of rows for each dataset
print("Number of rows in noc_regions.csv: ", noc_regions_data.shape[0])
print("Number of rows in regions_iso.csv: ", iso_data.shape[0])
print("Number of rows in nocs_list.csv: ", nocs_list.shape[0])

Number of rows in noc_regions.csv:  230
Number of rows in regions_iso.csv:  252
Number of rows in nocs_list.csv:  209


In [3]:
# Merge the dataframes using an outer join
merged_data = pd.merge(noc_regions_data, nocs_list, on="NOC", how="outer")
merged_data.drop(columns=["notes"], inplace=True)
merged_data

,NOC,region,continent,name
0,AFG,Afghanistan,Asia,Afghanistan
1,AHO,Curaçao,NaN,NaN
2,ALB,Albania,Europe,Albania
3,ALG,Algeria,Africa,Algeria
4,AND,Andorra,Europe,Andorra
...,...,...,...,...
228,YEM,Yemen,Asia,Yemen
229,YMD,Yemen,NaN,NaN
230,YUG,Serbia,NaN,NaN
231,ZAM,Zambia,Africa,Zambia


In [4]:
# set 'name' the same value as 'region' if 'name' is NaN
for i in range(merged_data.shape[0]):
    if pd.isnull(merged_data.loc[i, "name"]):
        merged_data.loc[i, "name"] = merged_data.loc[i, "region"]


In [5]:
# get all entries with nan values in name
nan_entries = merged_data[merged_data["name"].isna()]
nan_entries


,NOC,region,continent,name
171,ROT,NaN,NaN,NaN
216,UNK,NaN,NaN,NaN


In [6]:
# set 'name' as 'N/A' for all entries with NaN values in name
for i in range(merged_data.shape[0]):
    if pd.isnull(merged_data.loc[i, "name"]):
        merged_data.loc[i, "name"] = "N/A"


In [7]:
# Merge the dataframes using an outer join
merged_data_iso = pd.merge(
    merged_data, iso_data, left_on="region", right_on="name", how="outer"
)
# Unify 'name_x' and 'name_y' into a single 'name' column
merged_data_iso["name"] = merged_data_iso["name_x"].combine_first(
    merged_data_iso["name_y"]
)

# Drop the original 'name_x' and 'name_y' columns
merged_data_iso = merged_data_iso.drop(["name_x", "name_y"], axis=1)

merged_data_iso

,NOC,region,continent,ISO,name
0,AFG,Afghanistan,Asia,AFG,Afghanistan
1,ALB,Albania,Europe,ALB,Albania
2,ALG,Algeria,Africa,DZA,Algeria
3,ASA,American Samoa,Oceania,ASM,American Samoa
4,AND,Andorra,Europe,AND,Andorra
...,...,...,...,...,...
273,EOR,NaN,NaN,NaN,IOC Refugee Olympic Team
274,IOP,NaN,NaN,NaN,Independent Olympic Participants
275,LBN,NaN,Asia,NaN,Lebanon
276,ROT,NaN,NaN,NaN,N/A


In [8]:
import numpy as np

# Fill missing values in 'name' column with a placeholder
merged_data_iso["name"] = merged_data_iso["name"].fillna("missing")

# Fill missing values for the same country with the existing values (except for NOC)
merged_data_iso = merged_data_iso.groupby("name").apply(
    lambda group: group.ffill().bfill()
)

# Replace the placeholder with np.nan
merged_data_iso["name"] = merged_data_iso["name"].replace("missing", np.nan)

# Reset the index
merged_data_iso = merged_data_iso.reset_index(drop=True)

C:\Users\alvarog\AppData\Local\Temp\ipykernel_15920\2822348710.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda group: group.ffill().bfill()
C:\Users\alvarog\AppData\Local\Temp\ipykernel_15920\2822348710.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_data_iso = merged_data_iso.groupby("name").apply(


In [9]:
merged_data_iso.drop(columns=["region"], inplace=True)

In [10]:
merged_data_iso

,NOC,continent,ISO,name
0,AFG,Asia,AFG,Afghanistan
1,ALB,Europe,ALB,Albania
2,ALG,Africa,DZA,Algeria
3,ASA,Oceania,ASM,American Samoa
4,AND,Europe,AND,Andorra
...,...,...,...,...
273,YMD,Asia,YEM,Yemen
274,ZAM,Africa,ZMB,Zambia
275,RHO,Africa,ZWE,Zimbabwe
276,ZIM,Africa,ZWE,Zimbabwe


In [11]:
continentes = {
    "Anguilla": "Americas",
    "Antarctica": "Antarctica",
    "Bonaire, Sint Eustatius and Saba": "Americas",
    "Bouvet Island": "Antarctica",
    "British Indian Ocean Territory": "Asia",
    "Christmas Island": "Oceania",
    "Cocos (Keeling) Islands": "Asia",
    "Curaçao": "Americas",
    "Czechia": "Europe",
    "Falkland Islands (Malvinas)": "Americas",
    "Faroe Islands": "Europe",
    "French Guiana": "Americas",
    "French Polynesia": "Oceania",
    "French Southern Territories": "Antarctica",
    "Gibraltar": "Europe",
    "Greenland": "Americas",
    "Guadeloupe": "Americas",
    "Guernsey": "Europe",
    "Heard Island and McDonald Islands": "Antarctica",
    "Holy See": "Europe",
    "IOC Refugee Olympic Team": "No continent",
    "Independent Olympic Athletes": "No continent",
    "Independent Olympic Participants": "No continent",
    "Isle of Man": "Europe",
    "Jersey": "Europe",
    "Macao": "Asia",
    "Martinique": "Americas",
    "Mayotte": "Africa",
    "Montserrat": "Americas",
    "New Caledonia": "Oceania",
    "Niue": "Oceania",
    "Norfolk Island": "Oceania",
    "Northern Mariana Islands": "Oceania",
    "Pitcairn": "Oceania",
    "Réunion": "Africa",
    "Saint Barthélemy": "Americas",
    "Saint Helena, Ascension and Tristan da Cunha": "Africa",
    "Saint Martin (French part)": "Americas",
    "Saint Pierre and Miquelon": "Americas",
    "Sint Maarten (Dutch part)": "Americas",
    "South Georgia and the South Sandwich Islands": "Antarctica",
    "Svalbard and Jan Mayen": "Europe",
    "Tokelau": "Oceania",
    "Turks and Caicos Islands": "Americas",
    "United States Minor Outlying Islands": "Oceania",
    "Viet Nam": "Asia",
    "Wallis and Futuna": "Oceania",
    "Western Sahara": "Africa",
    "Åland Islands": "Europe",
}
# segun el diccionario continents, se asigna el continente correspondiente a cada pais
merged_data_iso["continent"] = merged_data_iso["continent"].fillna(
    merged_data_iso["name"].map(continentes)
)
merged_data_iso


,NOC,continent,ISO,name
0,AFG,Asia,AFG,Afghanistan
1,ALB,Europe,ALB,Albania
2,ALG,Africa,DZA,Algeria
3,ASA,Oceania,ASM,American Samoa
4,AND,Europe,AND,Andorra
...,...,...,...,...
273,YMD,Asia,YEM,Yemen
274,ZAM,Africa,ZMB,Zambia
275,RHO,Africa,ZWE,Zimbabwe
276,ZIM,Africa,ZWE,Zimbabwe


In [12]:
# get all entries with missing iso
missing_iso = merged_data_iso[merged_data_iso["ISO"].isna()]
missing_iso


,NOC,continent,ISO,name
114,EOR,No continent,NaN,IOC Refugee Olympic Team
116,IOA,No continent,NaN,Independent Olympic Athletes
117,IOP,No continent,NaN,Independent Olympic Participants
168,ROT,NaN,NaN,N/A
169,UNK,NaN,NaN,N/A


In [13]:
# set 'ISO' as 'N/A' for all entries with NaN values in ISO
for i in range(merged_data_iso.shape[0]):
    if pd.isnull(merged_data_iso.loc[i, "ISO"]):
        merged_data_iso.loc[i, "ISO"] = "N/A"


In [14]:
merged_data_iso.to_csv("datasets/iso_noc-merged.csv", index=False)